In [3]:
#构造数据集
import os
import paddle
from paddle.io import Dataset
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import json
#读取数据
def read_data(path):
    with open(path, "r") as infile:
        data=json.load(infile)
    return data
filepath = r"./work/geo_derived_feature1.0.json"  #dataset_dir为数据集实际路径，需要填写全路径
data=read_data(filepath)

In [9]:
postive=[ v for v in data if v['label']==0]
for v in postive[:100]:
    print(v)

{'source_name': 'Augusta Park', 'source_fclass': 'park', 'derived_name': 'Lincoln Park Postal Store', 'derived_fclass': 'post_office', 'topy_vec': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], 'near_vec': [1.0, 0.0], 'dif_start': [0.0, 1.0], 'dif_end': [1.0, 0.0], 'label': 0}
{'source_name': 'Park Summit Trail', 'source_fclass': 'residential', 'derived_name': 'Mountain Park Drive', 'derived_fclass': 'residential', 'topy_vec': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], 'near_vec': [1.0, 0.0], 'dif_start': [0.0, 1.0], 'dif_end': [0.0, 1.0], 'label': 0}
{'source_name': 'Schlagel Park', 'source_fclass': 'park', 'derived_name': 'Park Drive', 'derived_fclass': 'residential', 'topy_vec': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0], 'near_vec': [1.0, 0.0], 'dif_start': [1.0, 0.0], 'dif_end': [1.0, 0.0], 'label': 0}
{'source_name': 'Tampier Lake', 'source_fclass': 'water', 'derived_name': 'Oak Park Avenue', 'derived_fclass': 'residential', 'topy_vec': [0.0, 0.0, 0.0, 0.0, 

In [2]:
#添加地名形态拓扑关系
def add_name_topy(dataset):
    new_dataset=[]
    for v in dataset:
        topy_vec=v['topy_vec']
        near_vec=v['near_vec']
        dif_start=v['dif_start']
        dif_end=v['dif_end']
        source_fclass=v['source_fclass']
        derived_fclass=v['derived_fclass']
        source_name=v['source_name']
        derived_name=v['derived_name']
        label=v['label']
        #添加地名拓扑关系
        if source_name!=derived_name:
            name_topy=set(derived_name.split(' '))&set(source_name.split(' '))
            if len(name_topy)==0:   
                topy='Disjoint'
            elif name_topy==set(source_name.split(' ')):
                topy='Within'
            else:
                topy="Intersect"
        v['morphological_topology']=topy
        new_dataset.append(v)
    return new_dataset
new_dataset=add_name_topy(data)

In [3]:
#基于地学统计的通名派生关系实验
topy_list=["Intersect",'Crosses', 'Touches','Within','Contains','Overlaps','Disjoint']
near_list=['no_near','near']
dif_time=['<0','>0']
def predict(dataset):
    new_dataset=[]
    for v in dataset:
        topy_vec=v['topy_vec']
        near_vec=v['near_vec']
        dif_start=v['dif_start']
        dif_end=v['dif_end']
        source_fclass=v['source_fclass']
        derived_fclass=v['derived_fclass']
        source_name=v['source_name']
        derived_name=v['derived_name']
        label=v['label']
        #获取地名拓扑特征
        morphological_topy=v['morphological_topology']
        #获取拓扑特征
        topy=[ i for i in range(len(topy_vec)) if topy_vec[i]!=0]
        topy_word=[ topy_list[v] for v in topy]
        #获取邻近特征
        near=[ i for i in range(len(near_vec)) if near_vec[i]!=0]
        near_word=[ near_list[v] for v in near]
        #获取时间特征
        start=[ i for i in range(len(dif_start)) if dif_start[i]!=0 ]
        dif_start_sign=[ dif_time[i] for i in  start]
        end=[ i for i in range(len(dif_end)) if dif_end[i]!=0 ]
        dif_end_sign=[ dif_time[i] for i in  end]
        newlabel=0
        label=v['label']
        if source_name!=derived_name:
            #river->*派生模式
            if source_fclass=='river':
                derivedWord=['Riverview','Riverside','River',
                'Rivers', 'Riverwalk','Riverbank','Riverway',
                'Riverboat','Riverward']
                source_geo=['River','Rivers']
                topy=["Intersect",'Crosses', 'Touches','Disjoint']
                #规则1
                dif1=set(source_name.split(' '))&set(source_geo)
                dif2=set(derived_name.split(' '))&set(derivedWord)
                dif_topy=set(topy)&set(topy_word)
                if len(dif1)!=0 and len(dif2)!=0 and len(dif_topy)!=0 and near_word==['near'] and dif_start_sign==['<0'] and dif_end_sign==['>0']:
                    #地名拓扑关系
                    if morphological_topy!='Within':
                       newlabel=1
                v['label']=newlabel
            # #lalke->*
            if source_fclass=='lake':
                derivedWord=['Lakeview','Lakes','Lakeside','Lakeshore',\
                            'Laker','Lakefront','Lakeport',\
                            'Lake','Lakelet','Lakelets']
                source_geo=['Lake','Lakes']
                topy=["Intersect",'Crosses', 'Touches','Disjoint']
                #规则1
                dif1=set(source_name.split(' '))&set(source_geo)
                dif2=set(derived_name.split(' '))&set(derivedWord)
                dif_topy=set(topy)&set(topy_word)
                if len(dif1)!=0 and len(dif2)!=0 and len(dif_topy)!=0 and near_word==['near'] and dif_start_sign==['<0'] and dif_end_sign==['>0']:
                    #地名拓扑关系
                    if morphological_topy!='Within':
                       newlabel=1
                v['label']=newlabel
            # #park->*
            if source_fclass=='park':
                derivedWord=['Park','Parkland','Parks','Parkside']
                source_geo=['Park','Parks']
                topy=["Intersect",'Crosses', 'Touches','Within','Contains','Overlaps']
                #规则1
                dif1=set(source_name.split(' '))&set(source_geo)
                dif2=set(derived_name.split(' '))&set(derivedWord)
                dif_topy=set(topy)&set(topy_word)
                if len(dif1)!=0 and len(dif2)!=0 and len(dif_topy)!=0 and near_word==['near'] and dif_start_sign==['<0'] and  dif_end_sign==['>0']:
                    #地名拓扑关系
                    if morphological_topy!='Within':
                       newlabel=1
                v['label']=newlabel
            new_dataset.append(v)
    return  new_dataset
dataset=predict(new_dataset)

In [4]:
postive_dataset=[v for v in dataset if v['label']==1]
negtive_dataset=[v for v in dataset if v['label']==0]
print(len(postive_dataset))
print(len(negtive_dataset)) 

2620
6410


In [5]:
# #上采样
# num=int(len(negtive_dataset)/len(postive_dataset))
# postive_dataset1=num*postive_dataset
# print(len(postive_dataset1))
# n=len(negtive_dataset)-len(postive_dataset1)
# print(n)
# postive_sapmle2=random.sample(postive_dataset,n)
# postive_sapmle3=postive_sapmle2+postive_dataset1
# dataset=postive_sapmle3+negtive_dataset
# print(len(postive_sapmle3),len(negtive_dataset))
# print(len(dataset))

In [6]:
#构建时间关系负例
#构建时间关系负例
import random

negtive2=random.sample(postive_dataset,262)

negtive3=[]
for v in negtive2:
    random_number = random.random()
    v['label']=0
    if random_number>0.5:
       v['dif_start']=[0.0,1.0]
    else:
       v['dif_end']=[1.0,0.0]    
    negtive3.append(v)

In [7]:
#下采样
import random
negtive_dataset1=random.sample(negtive_dataset,2358)
print(len(negtive_dataset1))

2358


In [8]:
#拼接正样本与负样本
dataset=postive_dataset+negtive_dataset1+negtive3
print(len(dataset))

5240


In [9]:
# #下采样
# import random
# negtive_dataset1=random.sample(negtive_dataset,len(postive_dataset))
# dataset=negtive_dataset1+postive_dataset
# print(len(dataset))


In [10]:
#划分训练集与测试集
from sklearn.model_selection import train_test_split
#划分训练、验证、测试数据集
train_dataset,test_dataset=train_test_split(dataset,train_size=0.8,random_state=34,shuffle=True)
print('tran_dataset size',len(train_dataset),'val_dataset size',len(test_dataset))

# test_dataset=test_dataset
# print('test',len(test_dataset))


tran_dataset size 4192 val_dataset size 1048


In [11]:
#保存数据
import pickle
def save_data(filepath,data):
    with open(filepath,'wb') as f :
         pickle.dump(data,f)
    return
    
def read_data(filepath):
    with open(filepath,'rb') as f :
         data=pickle.load(f)
    return data
# filepath=r'/home/aistudio/work/geo_derived_feature.pickle' 
# save_data(filepath,train_dataset)
# filepath=r'/home/aistudio/work/test.pickle' 
# save_data(filepath,test_dataset)

In [12]:
 #基于语义空间关系标注数据
import random

#datalist=random.sample(datalist,2837)
def predict(data):
    predlist=[]
    count=0
    for v in data:
        topy_vec=v['topy_vec']
        near_vec=v['near_vec']
        dif_start=v['dif_start']
        dif_end=v['dif_end']
        source_fclass=v['source_fclass']
        derived_fclass=v['derived_fclass']
        source_name=v['source_name']
        derived_name=v['derived_name']
        label=v['label']
        #获取地名拓扑特征
        morphological_topy=v['morphological_topology']
        #获取拓扑特征
        topy=[ i for i in range(len(topy_vec)) if topy_vec[i]!=0]
        topy_word=[ topy_list[v] for v in topy]
        #获取邻近特征
        near=[ i for i in range(len(near_vec)) if near_vec[i]!=0]
        near_word=[ near_list[v] for v in near]
        #获取时间特征
        start=[ i for i in range(len(dif_start)) if dif_start[i]!=0 ]
        dif_start_sign=[ dif_time[i] for i in  start]
        end=[ i for i in range(len(dif_end)) if dif_end[i]!=0 ]
        dif_end_sign=[ dif_time[i] for i in  end]
        label=v['label']
        preds=0
        #river->*派生模式
        v['pred']=label
        if source_fclass=='river':
            derivedWord=['River']
            source_geo=['River']
            topy=["Intersect",'Crosses', 'Touches','Disjoint']
            #规则1
            dif1=set(source_name.split(' '))&set(source_geo)
            dif2=set(derived_name.split(' '))&set(derivedWord)
            dif_topy=set(topy)&set(topy_word)
            if len(dif1)!=0 and len(dif2)!=0 and len(dif_topy)!=0 and near_word==['near'] :
                    #地名拓扑关系
                    preds=1
            v['pred']=preds
        #lalke->*
        if source_fclass=='lake':
            derivedWord=['Lakes','Lake']
            source_geo=['Lake','Lakes']
            topy=["Intersect",'Crosses', 'Touches','Disjoint']
            #规则1
            dif1=set(source_name.split(' '))&set(source_geo)
            dif2=set(derived_name.split(' '))&set(derivedWord)
            dif_topy=set(topy)&set(topy_word)
            if len(dif1)!=0 and len(dif2)!=0 and len(dif_topy)!=0 and  near_word==['near'] :
                preds=1
            v['pred']=preds
        #park->*
        if source_fclass=='park':
            derivedWord=['Park']
            source_geo=['Park']
            topy=["Intersect",'Crosses', 'Touches','Within','Contains','Overlaps']
            #规则1
            dif1=set(source_name.split(' '))&set(source_geo)
            dif2=set(derived_name.split(' '))&set(derivedWord)
            dif_topy=set(topy)&set(topy_word)
            if len(dif1)!=0 and len(dif2)!=0 and len(dif_topy)!=0 and near_word==['near']:
                preds=1
            v['pred']=preds
        if label!=preds:
            count+=1
        predlist.append(v)
    print(count)
    return predlist
test_pred=predict(test_dataset)
print(len(test_dataset))


286
1048


In [13]:
labels=[ v['label'] for v in test_pred]
preds=[ v['pred'] for v in test_pred]


In [14]:
# #计算评价指标
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy=accuracy_score(labels, preds)
precision=precision_score(labels, preds, average='weighted')
recall=recall_score(labels, preds, average='weighted')
f1=f1_score(labels, preds, average='weighted') 
print(accuracy,precision,recall,f1)
metric={'accuracy':accuracy,'precision':precision,'recall':recall,'f1':f1}

0.7270992366412213 0.7512262626179403 0.7270992366412213 0.7258454446476095
